In [7]:
import requests
import json
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
outputs_url = "http://0.0.0.0:8000/api/v1/outputs_elastic/?substance=caffeine&pktype=clearance&final=true"
units_factor = {'ml/min/kg':60 ,'ml/min':np.NaN, 'ml/h/kg':1
units_mapping = {'ml/min/kg':'ml/h/kg','ml/min':None, 'ml/h/kg':'ml/h/kg'}                

In [12]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def norm_value(df_row):
    if df_row.value:
        value = df_row.value
    elif df_row.mean:
        value = df_row.mean
        
    factor = units_factor[df_row.unit]
    return value * factor
    

In [13]:
response = requests.get(outputs_url)

In [14]:
num_pages = response.json()["last_page"]

In [15]:
data = []
for page in range(2,num_pages +1):
    url = outputs_url + f"&page={page}"
    response = requests.get(url)
    data += response.json()["data"]["data"]

In [21]:
flatten_data = [flatten_json(d) for d in data]


In [22]:
df_outputs = pd.DataFrame(flatten_data)

In [23]:
df_outputs

,cv,final,group,group_name,group_pk,individual,individual_name,individual_pk,interventions_0_name,interventions_0_pk,...,pk,pktype,sd,se,substance_name,time,time_unit,tissue,unit,value
0,None,True,NaN,NaN,NaN,NaN,1C,55.0,Dcaf,2,...,125,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,1.74
1,None,True,NaN,NaN,NaN,NaN,2C,56.0,Dcaf,2,...,126,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,1.29
2,None,True,NaN,NaN,NaN,NaN,3C,57.0,Dcaf,2,...,127,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,1.06
3,None,True,NaN,NaN,NaN,NaN,4C,58.0,Dcaf,2,...,128,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,3.12
4,None,True,NaN,NaN,NaN,NaN,5C,59.0,Dcaf,2,...,129,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,0.95
5,None,True,NaN,NaN,NaN,NaN,6C,60.0,Dcaf,2,...,130,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,1.11
6,None,True,NaN,NaN,NaN,NaN,7C,61.0,Dcaf,2,...,131,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,3.27
7,None,True,NaN,NaN,NaN,NaN,8C,62.0,Dcaf,2,...,132,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,1.89
8,None,True,NaN,NaN,NaN,NaN,9C,63.0,Dcaf,2,...,133,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,2.00
9,None,True,NaN,NaN,NaN,NaN,10C,64.0,Dcaf,2,...,134,clearance,None,None,caffeine,NaN,None,plasma,ml/min/kg,2.05
